<a href="https://colab.research.google.com/github/sarahbanat/Greeting-application/blob/main/3D_point_cloud_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install open3d && touch /root/.ipython/profile_default/startup/00-first.py && echo "import open3d" >> /root/.ipython/profile_default/startup/00-first.py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.5/420.5 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.9/214.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.3 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.5
    Uninstalling widgetsnbextension-3.6.5:
      Successfully uninstalled widgetsnbextension-3.6.5
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 2.3.7
    Uninstalling Werkzeug-2.3.7:
      Successfully uninstalled Werkzeug-2.3

In [8]:
import cv2
import numpy as np
import open3d as o3d
from google.colab import files



In [10]:
image = cv2.imread('/content/Hyundai-Grand-i10-Nios-200120231541.jpg')
depth_image = cv2.imread('/content/Hyundai-Grand-i10-Nios-200120231541_depth.jpg', cv2.IMREAD_ANYDEPTH)


In [31]:
image = cv2.imread('/content/panorama_2.jpg')
depth_image = cv2.imread('/content/panorama_2_depth.jpg', cv2.IMREAD_ANYDEPTH)


In [38]:


depth_image_resized = cv2.resize(depth_image, (image.shape[1], image.shape[0]))


#smoothing filter (noise redutction)
#depth_image_resized = cv2.GaussianBlur(depth_image_resized, (5, 5), 0)


# Create an Open3D PointCloud object
ply = o3d.geometry.PointCloud()



In [39]:


# convert depth image to pc


'''
depth_image_resized is a 2D array where each element [i, j] -> the depth ( distance from camera)
of the corresponding pixel in the image.

depth_image_resized.shape returns a tuple (height, width) ->  dimensions of this 2D array.

h-> height of the image in pixels
w -> width in pixels
'''
h, w = depth_image_resized.shape


'''np.linspace(0, w-1, w) generates an array of evenly spaced values from 0 to w-1.
; creates a 1D array [0, 1, 2, ..., w-2, w-1]

np.linspace(0, h-1, h) does the same but for the height-> creating a 1D array [0, 1, 2, ..., h-2, h-1].

np.meshgrid() takes these two 1D arrays and produces two 2D arrays x and y such that x[i, j]
and y[i, j] will give you the x and y coordinates of the pixel
'''
x, y = np.meshgrid(np.linspace(0, w-1, w), np.linspace(0, h-1, h))

'''z is 2D array : contains the depth information
(z-coordinate) corresponding to each pixel in  di'''
z = depth_image_resized

'''flattening -> converts 2d arrays into 1d arrays
np.column_stack() -> takes these three 1d arrays and stacks them as columns in a new 2d array xyz like so:
x = [x1,x2,x3]
y = [y1,y2,y3]
z = [z1,z2,z3]
 ------------->
[[x1, y1, z1],
 [x2, y2, z2],
 [x3, y3, z3]]

'''
xyz = np.column_stack((x.flatten(), y.flatten(), z.flatten()))

# set pc coordinates
ply.points = o3d.utility.Vector3dVector(xyz)

# take colors from original images
colors = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
colors = colors.reshape(-1, 3) / 255.0  # Normalize to [0, 1]

# set pc colors
ply.colors = o3d.utility.Vector3dVector(colors)

####### 2 methods to flip pc othrwise it will be upside down#################
#the point cloud result is upside down: flip points on y-axis and z-axis ,
# a point (x,y,z) s transfromed to (x,-y,-z) (mirrored along the x-axis)

#points = np.asarray(ply.points)
#points[:, 1] = -points[:, 1]
#ply.points = o3d.utility.Vector3dVector(points)


ply.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

''' Open3D provides a function called voxel_down_sample to downsample a point cloud by creating a grid of voxels
over the point cloud and averaging the points that fall into each voxel'''
ply = ply.voxel_down_sample(voxel_size=0.1)


#outlier removal (fill 0s)
#ply, ind = ply.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)



In [40]:

o3d.io.write_point_cloud("output_point_cloud.ply", ply)
files.download("output_point_cloud.ply")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>